In [77]:
!pip install numpy==1.24.3 scipy==1.13.1 gensim underthesea #--no-cache-dir #--force-reinstall


In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [79]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [80]:
import numpy as np
import pandas as pd
import underthesea
import gensim
print("✅ Numpy version:", np.__version__)
print("✅ Underthesea test:", underthesea.word_tokenize("Tôi yêu tiếng Việt."))

✅ Numpy version: 1.24.3
✅ Underthesea test: ['Tôi', 'yêu', 'tiếng', 'Việt', '.']


In [81]:
STOP_WORD_FILE = '/content/gdrive/MyDrive/Colab Notebooks/Capstone/Pro2/Data/vietnamese-stopwords.txt'

In [82]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [83]:
# đọc tập tin sample_products.csv vào df
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Capstone/Pro2/Data/Products_ThoiTrangNam_raw.csv")
df.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g%C3%A2n-form-body-t%C3%B4n-d%C3%A1ng-i.83496725.6939920023?sp_atk=887ef2fb-5f8a-43ac-99b7-3267848ec9fc,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdfaa1d5d327f95_tn,86250.0,4.9,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nChiều dài tay áo\nKhác\nPhong cách\nThể thao, Cơ bản, Đường phố, Nhiệt đới\nTall Fit\nCó\nXuất xứ\nViệt Nam\nDáng kiểu áo\nÔm sát\nChất liệu\nCotton\nMẫu\nSọc tăm có gân nổi\nKho hàng\n75317\nGửi từ\nBà Rịa - Vũng Tàu\n\nBEST TANK TOPS FOR MEN 2021\n\nÁo Ba Lỗ Quốc Dân Cho Nam \n—————————————————\nBảng size :\nSize M : 45kg-50kg\nSize L : 55kg -60kg \nSize XL : 65kg- 70kg\nSize XXL : 75kg - 80kg \n\n(Bảng size này là chuẩn mặc kiểu body nha quý khách còn nếu khách nào không thích mặc ôm thì tăng lên 1 size nhé còn muốn mặc rộng rộng thì tăng lên 2 size \nbảng size chỉ mang tính chất tham khảo vì có nhiều khách thể hình không được cân đối .Để được tư vấn chính xác vui lòng inbox cho shop nhé)\n\nKích thước :\nSize M: dài 66cm Rộng :39cm (vòng ngực 78cm)\nSize L: dài 68cm. Rộng : 43cm (vòng ngực 86cm)\nSize XL:dài 70cm Rộng : 47cm (vòng ngực 94cm)\nSize 2XL: dài 72cm Rộng : 51cm (vòng ngực 102cm)\n\nVòng nách : \nM : dài 25cm\nL : dài 27cm\nXL: dài 29cm\nXXL : dài 31cm\n\nBảng size trên là mặc vừa cơ thể (không quá ôm sát cũng không bị rộng quá vừa vừa người) tuỳ vào sở thích mỗi người nếu bạn có body chuẩn 6 múi muốn mặc ôm sát thì hạ xuống 1 size , nếu muốn mặc rộng rãi thoải mái thì tăng thêm 1 size nhé!\nBảng size chỉ mang tính chất tham khảo nó chỉ tương đối chứ không chính xác tuyệt đối nha khách vì phụ thuộc vào cơ địa mỗi người khác nhau . Mọi trường hợp thắc mắc về size sản phẩm , quý khách vui lòng nhắn tin cho shop để được tư vấn chính xác nhất! \n\nÁo ba lỗ thun gân ,một mẫu áo basic thiết yếu và tiện lợi, hầu như được tất cả phái mạnh trên toàn thế giới đều rất ưa chuộng , cực kỳ yêu thích và tất nhiên các chàng trai Việt Nam của chúng ta cũng không ngoại lệ !\n\nNgoài những ưu điểm chính như: trẻ trung năng động , không lỗi thời , đơn giản dễ mặc , chất vải thun cotton thấm hút tốt , độ co giãn đàn hồi cao những sợi gân vải nổi giúp tăng thêm sự cá tính và khỏe khoán cho người mặc … thì điểm nhấn đó chính là form áo cực đẹp tôn vinh lên vóc dáng ,giúp những chàng trai trở nên tự tin hơn cuốn hút hơn! \n\nCó thể mix với áo khoác , áo vét …mặc ở nhà , đi chơi , đi gym, du lịch ..\n\nchất vải Shop xin cam kết giống như hình ảnh Shop đăng bán , vải đẹp và đanh , cứng fom, không nhão , không chảy , không bèo nhèo …\nkhi nhận hàng nếu sản phẩm không giống mô tả quý khách được quyền bấm trả hàng hoàn tiền !"
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr%E1%BA%AFng-Ch%E1%BA%A5t-Cotton-Si%C3%AAu-M%C3%A1t-Si%C3%AAu-%C4%90%E1%BA%B9p-i.170393318.2662407578?sp_atk=1c63002b-43ec-41f4-b681-b6d1b8cc6d5f,https://cf.shopee.vn/file/c7ea4c6574dc79be6b266b9d69b49abc_tn,26800.0,4.9,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXuất xứ\nViệt Nam\nTên tổ chức chịu trách nhiệm sản xuất\nAzman\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nPhú Lãm - Hà Đông - Hà Nội\nKho hàng\n61911\nGửi từ\nHà Nội\n\nChất liệu 100 % cotton ( vải bông) đem đến sự mềm mại, thoáng khí, thấm hút mổ hôi tốt, giúp bạn khi mặc sẽ luôn cảm thấy thỏa mái và tự tin.\n- Co giãn tốt, không gây vết lằn trên da, không gây sự khó chịu khi mặc.\n- Áo được thiết kế đơn giản, dễ mặc cho nam giới phong cách năng động và khỏe khoắn hơn.\n- Sản phẩm được sản xuất tại việt nam, an toàn và không gây hại tới sức khỏe\n#aolot #aonot #aothun #aothunin #aothuntrang #aobalo #ao3lo #aophongbalo #aothunhanquoc #aothuncotron #aothunnam #aothunnamdep #aothundongphuc#aothuntron #aothundep"
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không xù, mềm mịn",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%

In [84]:
df_col= pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Capstone/Pro2/Data/Products_ThoiTrangNam_rating_raw.csv")
df_col.head(10)

,product_id\tuser_id\tuser\trating
0,190\t1\tkarmakyun2nd\t5
1,190\t2\ttranquangvinh_vv\t5
2,190\t3\tnguyenquoctoan2005\t5
3,190\t4\tnguyenthuyhavi\t5
4,190\t5\tluonganh5595\t5
5,190\t6\thong_son0509\t5
6,190\t7\tduylee1103\t5
7,190\t8\tx*****5\t5
8,190\t9\ttinotrip02\t5
9,190\t10\tlechitam14062002\t5


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49663 entries, 0 to 49662
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    49663 non-null  int64  
 1   product_name  49663 non-null  object 
 2   category      49663 non-null  object 
 3   sub_category  49663 non-null  object 
 4   link          49663 non-null  object 
 5   image         36443 non-null  object 
 6   price         49663 non-null  float64
 7   rating        49663 non-null  float64
 8   description   48700 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.4+ MB


In [86]:
df.description.head(3)

,description
0,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nChiều dài tay áo\nKhác\nPhong cách\nThể thao, Cơ bản, Đường phố, Nhiệt đới\nTall Fit\nCó\nXuất xứ\nViệt Nam\nDáng kiểu áo\nÔm sát\nChất liệu\nCotton\nMẫu\nSọc tăm có gân nổi\nKho hàng\n75317\nGửi từ\nBà Rịa - Vũng Tàu\n\nBEST TANK TOPS FOR MEN 2021\n\nÁo Ba Lỗ Quốc Dân Cho Nam \n—————————————————\nBảng size :\nSize M : 45kg-50kg\nSize L : 55kg -60kg \nSize XL : 65kg- 70kg\nSize XXL : 75kg - 80kg \n\n(Bảng size này là chuẩn mặc kiểu body nha quý khách còn nếu khách nào không thích mặc ôm thì tăng lên 1 size nhé còn muốn mặc rộng rộng thì tăng lên 2 size \nbảng size chỉ mang tính chất tham khảo vì có nhiều khách thể hình không được cân đối .Để được tư vấn chính xác vui lòng inbox cho shop nhé)\n\nKích thước :\nSize M: dài 66cm Rộng :39cm (vòng ngực 78cm)\nSize L: dài 68cm. Rộng : 43cm (vòng ngực 86cm)\nSize XL:dài 70cm Rộng : 47cm (vòng ngực 94cm)\nSize 2XL: dài 72cm Rộng : 51cm (vòng ngực 102cm)\n\nVòng nách : \nM : dài 25cm\nL : dài 27cm\nXL: dài 29cm\nXXL : dài 31cm\n\nBảng size trên là mặc vừa cơ thể (không quá ôm sát cũng không bị rộng quá vừa vừa người) tuỳ vào sở thích mỗi người nếu bạn có body chuẩn 6 múi muốn mặc ôm sát thì hạ xuống 1 size , nếu muốn mặc rộng rãi thoải mái thì tăng thêm 1 size nhé!\nBảng size chỉ mang tính chất tham khảo nó chỉ tương đối chứ không chính xác tuyệt đối nha khách vì phụ thuộc vào cơ địa mỗi người khác nhau . Mọi trường hợp thắc mắc về size sản phẩm , quý khách vui lòng nhắn tin cho shop để được tư vấn chính xác nhất! \n\nÁo ba lỗ thun gân ,một mẫu áo basic thiết yếu và tiện lợi, hầu như được tất cả phái mạnh trên toàn thế giới đều rất ưa chuộng , cực kỳ yêu thích và tất nhiên các chàng trai Việt Nam của chúng ta cũng không ngoại lệ !\n\nNgoài những ưu điểm chính như: trẻ trung năng động , không lỗi thời , đơn giản dễ mặc , chất vải thun cotton thấm hút tốt , độ co giãn đàn hồi cao những sợi gân vải nổi giúp tăng thêm sự cá tính và khỏe khoán cho người mặc … thì điểm nhấn đó chính là form áo cực đẹp tôn vinh lên vóc dáng ,giúp những chàng trai trở nên tự tin hơn cuốn hút hơn! \n\nCó thể mix với áo khoác , áo vét …mặc ở nhà , đi chơi , đi gym, du lịch ..\n\nchất vải Shop xin cam kết giống như hình ảnh Shop đăng bán , vải đẹp và đanh , cứng fom, không nhão , không chảy , không bèo nhèo …\nkhi nhận hàng nếu sản phẩm không giống mô tả quý khách được quyền bấm trả hàng hoàn tiền !"
1,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXuất xứ\nViệt Nam\nTên tổ chức chịu trách nhiệm sản xuất\nAzman\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nPhú Lãm - Hà Đông - Hà Nội\nKho hàng\n61911\nGửi từ\nHà Nội\n\nChất liệu 100 % cotton ( vải bông) đem đến sự mềm mại, thoáng khí, thấm hút mổ hôi tốt, giúp bạn khi mặc sẽ luôn cảm thấy thỏa mái và tự tin.\n- Co giãn tốt, không gây vết lằn trên da, không gây sự khó chịu khi mặc.\n- Áo được thiết kế đơn giản, dễ mặc cho nam giới phong cách năng động và khỏe khoắn hơn.\n- Sản phẩm được sản xuất tại việt nam, an toàn và không gây hại tới sức khỏe\n#aolot #aonot #aothun #aothunin #aothuntrang #aobalo #ao3lo #aophongbalo #aothunhanquoc #aothuncotron #aothunnam #aothunnamdep #aothundongphuc#aothuntron #aothundep"
2,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nThương hiệu\nTyasuo\nChiều dài tay áo\nKhông tay\nPhong cách\nThể thao, Cơ bản, Đường phố\nTall Fit\nCó\nXuất xứ\nViệt Nam\nDáng kiểu áo\nRộng\nMẫu\nTrơn\nRất lớn\nCó\nChất liệu\nThun TC\nTên tổ chức chịu trách nhiệm sản xuất\nTyasuo Shop\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nP.Long Trường, Tp.Thủ Đức, Hồ Chí Minh\nKho hàng\n591\nGửi từ\nTP. Hồ Chí Minh\n\nUpdate 04/2022 : Từ góp ý của khách hàng trong năm 2021 shop đã cải thiện hoàn toàn chất vải và form áo, hiện tại sản phẩm đã chất lượng: ""Mặc mát, vải mềm mịn và đặt biệt KHÔNG NHÃO, KHÔNG XÙ, hầu như không nhăn ""\n\nThành phần :\n• Chất liệu thun TC: Cotton 65% cotton, 35 % polyester và pha chút Spandex để áo có độ co dãn ngang\n• Áo có cổ viền nhỏ như ảnh mô tả. \n• Form áo thoải mái có big size\

In [87]:

# Tăng độ dài tối đa để pandas hiển thị toàn bộ nội dung chuỗi
pd.set_option('display.max_colwidth', None)

df.description.head(5)


,description
0,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nChiều dài tay áo\nKhác\nPhong cách\nThể thao, Cơ bản, Đường phố, Nhiệt đới\nTall Fit\nCó\nXuất xứ\nViệt Nam\nDáng kiểu áo\nÔm sát\nChất liệu\nCotton\nMẫu\nSọc tăm có gân nổi\nKho hàng\n75317\nGửi từ\nBà Rịa - Vũng Tàu\n\nBEST TANK TOPS FOR MEN 2021\n\nÁo Ba Lỗ Quốc Dân Cho Nam \n—————————————————\nBảng size :\nSize M : 45kg-50kg\nSize L : 55kg -60kg \nSize XL : 65kg- 70kg\nSize XXL : 75kg - 80kg \n\n(Bảng size này là chuẩn mặc kiểu body nha quý khách còn nếu khách nào không thích mặc ôm thì tăng lên 1 size nhé còn muốn mặc rộng rộng thì tăng lên 2 size \nbảng size chỉ mang tính chất tham khảo vì có nhiều khách thể hình không được cân đối .Để được tư vấn chính xác vui lòng inbox cho shop nhé)\n\nKích thước :\nSize M: dài 66cm Rộng :39cm (vòng ngực 78cm)\nSize L: dài 68cm. Rộng : 43cm (vòng ngực 86cm)\nSize XL:dài 70cm Rộng : 47cm (vòng ngực 94cm)\nSize 2XL: dài 72cm Rộng : 51cm (vòng ngực 102cm)\n\nVòng nách : \nM : dài 25cm\nL : dài 27cm\nXL: dài 29cm\nXXL : dài 31cm\n\nBảng size trên là mặc vừa cơ thể (không quá ôm sát cũng không bị rộng quá vừa vừa người) tuỳ vào sở thích mỗi người nếu bạn có body chuẩn 6 múi muốn mặc ôm sát thì hạ xuống 1 size , nếu muốn mặc rộng rãi thoải mái thì tăng thêm 1 size nhé!\nBảng size chỉ mang tính chất tham khảo nó chỉ tương đối chứ không chính xác tuyệt đối nha khách vì phụ thuộc vào cơ địa mỗi người khác nhau . Mọi trường hợp thắc mắc về size sản phẩm , quý khách vui lòng nhắn tin cho shop để được tư vấn chính xác nhất! \n\nÁo ba lỗ thun gân ,một mẫu áo basic thiết yếu và tiện lợi, hầu như được tất cả phái mạnh trên toàn thế giới đều rất ưa chuộng , cực kỳ yêu thích và tất nhiên các chàng trai Việt Nam của chúng ta cũng không ngoại lệ !\n\nNgoài những ưu điểm chính như: trẻ trung năng động , không lỗi thời , đơn giản dễ mặc , chất vải thun cotton thấm hút tốt , độ co giãn đàn hồi cao những sợi gân vải nổi giúp tăng thêm sự cá tính và khỏe khoán cho người mặc … thì điểm nhấn đó chính là form áo cực đẹp tôn vinh lên vóc dáng ,giúp những chàng trai trở nên tự tin hơn cuốn hút hơn! \n\nCó thể mix với áo khoác , áo vét …mặc ở nhà , đi chơi , đi gym, du lịch ..\n\nchất vải Shop xin cam kết giống như hình ảnh Shop đăng bán , vải đẹp và đanh , cứng fom, không nhão , không chảy , không bèo nhèo …\nkhi nhận hàng nếu sản phẩm không giống mô tả quý khách được quyền bấm trả hàng hoàn tiền !"
1,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXuất xứ\nViệt Nam\nTên tổ chức chịu trách nhiệm sản xuất\nAzman\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nPhú Lãm - Hà Đông - Hà Nội\nKho hàng\n61911\nGửi từ\nHà Nội\n\nChất liệu 100 % cotton ( vải bông) đem đến sự mềm mại, thoáng khí, thấm hút mổ hôi tốt, giúp bạn khi mặc sẽ luôn cảm thấy thỏa mái và tự tin.\n- Co giãn tốt, không gây vết lằn trên da, không gây sự khó chịu khi mặc.\n- Áo được thiết kế đơn giản, dễ mặc cho nam giới phong cách năng động và khỏe khoắn hơn.\n- Sản phẩm được sản xuất tại việt nam, an toàn và không gây hại tới sức khỏe\n#aolot #aonot #aothun #aothunin #aothuntrang #aobalo #ao3lo #aophongbalo #aothunhanquoc #aothuncotron #aothunnam #aothunnamdep #aothundongphuc#aothuntron #aothundep"
2,"Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nThương hiệu\nTyasuo\nChiều dài tay áo\nKhông tay\nPhong cách\nThể thao, Cơ bản, Đường phố\nTall Fit\nCó\nXuất xứ\nViệt Nam\nDáng kiểu áo\nRộng\nMẫu\nTrơn\nRất lớn\nCó\nChất liệu\nThun TC\nTên tổ chức chịu trách nhiệm sản xuất\nTyasuo Shop\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nP.Long Trường, Tp.Thủ Đức, Hồ Chí Minh\nKho hàng\n591\nGửi từ\nTP. Hồ Chí Minh\n\nUpdate 04/2022 : Từ góp ý của khách hàng trong năm 2021 shop đã cải thiện hoàn toàn chất vải và form áo, hiện tại sản phẩm đã chất lượng: ""Mặc mát, vải mềm mịn và đặt biệt KHÔNG NHÃO, KHÔNG XÙ, hầu như không nhăn ""\n\nThành phần :\n• Chất liệu thun TC: Cotton 65% cotton, 35 % polyester và pha chút Spandex để áo có độ co dãn ngang\n• Áo có cổ viền nhỏ như ảnh mô tả. \n• Form áo thoải mái có big size\

In [88]:
df.description.tail(5)


,description
49658,"Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChiều dài vớ\nBắp chân\nKho hàng\n44\nGửi từ\nHuyện Thanh Liêm, Hà Nam\n\nTất vớ Nam cổ cao cao cấp, Tất vớ đi giày công sở là loại vớ với tông màu nhẹ nhàng rất dễ để phối với giày tây công sở, chất liệu coton khử mùi thoáng khí, êm ái khi sử dụng.\n\nĐẶC ĐIỂM NỔI BẬT\n\n– Mềm mại, thấm hút mồ hôi mang lại cảm giác mát mẻ khô thoáng suốt 4 mùa\n\n– Chất liệu cotton cao cấp có độ đàn hồi cực cao\n\n– Sản phẩm mịn màng, không bị xù lông khi dùng\n\n– Đa dạng sản phẩm rất dễ dàng phối đồ\n\nVÌ SAO BẠN CHỌN CHÚNG TÔI\n\n– Chúng tôi luôn lựa chọn sản phẩm với chất liệu tốt nhất, hình ảnh đúng như mô tả\nHotline: 0961650000\n\nLƯU Ý: KHÁCH IB ĐỂ CHỌN MẪU\n#tatnam #tatvonam #tatcaoconam #dochoigo.vn"
49659,"Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nThương hiệu\nLivan Sport\nChất liệu\nSợi tổng hợp\nXuất xứ\nViệt Nam\nChiều dài vớ\nMắt cá\nLoại tất/vớ\nVớ thể thao\nKho hàng\n2\nGửi từ\nHà Nội\n\nTẤT THỂ THAO CHẤT LIỆU BAMBOO Livan với ưu điểm vượt trội:\n✪ THÔNG TIN SẢN PHẨM\n- Chất liệu: mát, êm nhẹ, mềm mịn, thoáng khí \n- Thấm hút mồ hôi tốt \n- Kháng khuẩn đến 99,8% \n- Cách nhiệt, chống UV\n- Công nghệ dệt với đệm xù tăng cường ở bàn chân chống phồng rộp\n- Băng vòng ôm cố định bàn chân\n- Lớp silicon phía sau tránh tụt gót\n- Hỗ trợ vận động thoải mái và là người bạn đồng hành của phái mạnh mỗi ngày. \n- Mã sản phẩm: LV19215300\n- Màu sắc: Xanh Rêu Viền Đen | Trắng Viền Đen\n- Xuất sứ: Việt Nam\n\n✪ HƯỚNG DẪN BẢO QUẢN\n- Sản phẩm giặt tay nhiệt độ dưới 40 độ C\n- Không ngâm trong thuốc tẩy\n- Không giặt chung với quần áo khác.\n\n✪ LIVAN SPORT CAM KẾT\n- Sản phẩm hoàn toàn giống ảnh chụp\n- Sản phẩm được kiểm tra cẩn thận trước khi đóng gói và giao đến khách hàng.\n- Hàng có sẵn, giao hàng ngay khi nhận được đơn \n------\n#Livansport #thoitrangthethao #Livan #dotaplivan #TAT #tatnam #tatkhangkhuan #tatthethao #tatnamthethao #tatthethaonam tatlivansport"
49660,"Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChất liệu\nCotton\nXuất xứ\nTrung Quốc\nKho hàng\n2935\nGửi từ\nHuyện Hoài Đức, Hà Nội\n\nCombo 3 Đôi Tất ( Vớ ) Hài Chống Hôi Chân Uni hàng xịn\n-Tất hài được dệt bằng 95% Cotton Bamboo, kết hợp Nano phân tử Bạc giúp phòng chống mùi hôi, thoáng khí, thấm hút mồ hôi tốt. \n- Khử mùi hôi: với công nghệ Nano Phân Tử Bạc giúp diệt khuẩn, khử mùi hôi.\n- Công nghệ dệt thông minh kết hợp giữa lỗ thoáng và sợi cotton bamboo giúp cho chân thoáng khí, êm ái, chống bai và không bị hằn ngứa.\n- Gót tất được dệt cao để chống tuột gót\n- Thành phần 95% Cotton Bamboo (sợi tre), mềm mại, thoáng khí và thấm hút mồ hôi tốt giúp kháng khuẩn và chống mùi hôi.\n- Màu sắc: Đa dạng, phối mầu đẹp mắt, sang trọng.\n- Mầu sắc : 4 mầu : Đen - Ghi Đậm - Ghi Nhạt"
49661,"Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nKho hàng\n189\nGửi từ\nHà Nội\n\nTất khử mùi hôi cho nam với chất liệu từ vải thấm mồ hôi kèm theo khử mùi các bạn nam có thể tự tin vận động mà không lo bị mùi hôi.\nHàng chất lượng\nTất đi êm chân, mềm, thoáng mát.\n#hop10doitat #tat #bittat #vo #chonghoichan #tatchonghoichan #tatcocao #tatcongan #tatnam #tatnu #chongthoi"
49662,"Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nXuất xứ\nViệt Nam\nKho hàng\n494\nGửi từ\nHà Nội\n\nTất lười nam nữ - Giá Hủy Diệt\n\n- Chấy liệu cotton thấm mồ hôi và không thối chân\n- Có lót silicon chống tụt gót\n- Size giày từ 37 - 42 đi ok\n- Form vớ co dãn tốt , thấm hút mồ hôi\n- UNISEX nên phù hợp cho cả nam và nữ \n- Dưới gót lót silicon chống tụt gót\n\nCHÚ Ý :\n- Không nên chà xát mạnh bằng bàn chải\n- Nếu là vớ trắng nên tách ra giặt riêng và sử dụng giặt bằng dầu gội\n- Khuyến khích nên giặt bằng tay\n====================================\n🔸 Contact : 094.661.0246\n🔸 Số 33 Liền kề 9 KĐT Đại Thanh, Thanh Trì, Hà Nội\n#vớ #votanghinh #tattanghinh #tatluoi #tattanghinh"


In [89]:
# !pip install ydata_profiling -q

# import pandas as pd
# from ydata_profiling import ProfileReport


# # Tạo báo cáo EDA
# profile = ProfileReport(df, title="EDA tổng quát", explorative=True)
# profile.to_file("eda_recommender.html")

In [90]:
# from google.colab import files
# files.download("eda_recommender.html")


In [91]:

# Drop rows with missing values in the 'description' column
df.dropna(subset=['description'], inplace=True)


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48700 entries, 0 to 49662
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    48700 non-null  int64  
 1   product_name  48700 non-null  object 
 2   category      48700 non-null  object 
 3   sub_category  48700 non-null  object 
 4   link          48700 non-null  object 
 5   image         35542 non-null  object 
 6   price         48700 non-null  float64
 7   rating        48700 non-null  float64
 8   description   48700 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.7+ MB


In [93]:
df.product_id.value_counts().sort_values(ascending=False)

,count
product_id,
11860,2
11829,2
11824,2
11833,2
11857,2
...,...
175066,1
175046,1
175045,1


In [94]:
df2=df[df['rating'] > 0]


In [95]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31699 entries, 0 to 49662
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    31699 non-null  int64  
 1   product_name  31699 non-null  object 
 2   category      31699 non-null  object 
 3   sub_category  31699 non-null  object 
 4   link          31699 non-null  object 
 5   image         21132 non-null  object 
 6   price         31699 non-null  float64
 7   rating        31699 non-null  float64
 8   description   31699 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.4+ MB


In [96]:
df2.isnull().sum()

,0
product_id,0
product_name,0
category,0
sub_category,0
link,0
image,10567
price,0
rating,0
description,0


In [97]:
# prompt: drop các giá trị null

# Drop rows with missing values in all columns
df2.dropna(inplace=True)

df2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 21132 entries, 0 to 49662
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    21132 non-null  int64  
 1   product_name  21132 non-null  object 
 2   category      21132 non-null  object 
 3   sub_category  21132 non-null  object 
 4   link          21132 non-null  object 
 5   image         21132 non-null  object 
 6   price         21132 non-null  float64
 7   rating        21132 non-null  float64
 8   description   21132 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.6+ MB


<ipython-input-97-bb8f40d02624>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


In [98]:
df=df2.copy()

In [99]:
import re

# Hàm kiểm tra từ có phải là từ tiếng Việt "sạch"
def is_valid_vietnamese(word):
    vietnamese_chars = (
        "a-zA-Z0-9_"
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    pattern = f'^[{vietnamese_chars}]+$'
    return re.match(pattern, word) is not None

In [100]:
from collections import Counter
import re

def get_top_words(text_series, top_n=30):
    # Gộp toàn bộ nội dung description
    all_text = ' '.join(text_series.dropna()).lower()

    # Loại bỏ ký tự đặc biệt, giữ lại chữ cái & tiếng Việt
    all_text = re.sub(r'[^a-zA-Zà-ỹÀ-Ỹ0-9\s]', ' ', all_text)

    # Tách từ
    words = all_text.split()

    # Đếm tần suất
    word_counts = Counter(words)

    return word_counts.most_common(top_n)


In [101]:
clear_text=get_top_words(df['description'], top_n=20)


In [102]:
print(clear_text)

[('hàng', 93756), ('nam', 79622), ('sản', 70635), ('phẩm', 57736), ('áo', 52554), ('không', 50657), ('chất', 49875), ('size', 48524), ('và', 47738), ('có', 45092), ('từ', 41672), ('trang', 39850), ('thời', 36523), ('shop', 36203), ('cho', 36183), ('bạn', 34804), ('với', 34400), ('được', 31581), ('màu', 31198), ('tất', 31072)]


In [103]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_ngrams(corpus, ngram_range=(2, 2), top_n=20):
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    sorted_words = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return sorted_words[:top_n]
clear_ngram=get_top_ngrams(df['description'].fillna(''), ngram_range=(2, 2), top_n=10)
print(clear_ngram)

[('sản phẩm', 57223), ('thời trang', 30006), ('chất liệu', 27461), ('trang nam', 21359), ('danh mục', 20549), ('shopee thời', 20429), ('mục shopee', 20416), ('kho hàng', 20332), ('gửi từ', 20118), ('thể thao', 13712)]


In [104]:
import re

def clean_description(text):


    # Loại bỏ ký tự đặc biệt và dấu xuống dòng
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9à-ỹÀ-Ỹ\s]', ' ', text)  # Giữ lại tiếng Việt
# Chuyển thành chữ thường
    text = text.str.lower()
    # Loại bỏ nhiều khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    # Xoá các hashtag và từ khóa không cần thiết
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()


    patterns_to_remove = [
        r'danh mục.*?shopee',
        r'shopee',
        r'thời trang nam',
        r'sản phẩm',
        r'chất liệu',
        r'kho hàng',
        r'thời trang',
        r'gửi từ.*?(việt nam)?',
        r'kho hàng',
        r'xuất xứ',
        r'xứ việt',
        r'tên tổ chức.*?trách nhiệm',
        r'sản xuất tại.*',
        r'việt nam'
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)


    return text


In [105]:
# Hàm xử lý một mô tả
def filter_vietnamese_words(text):
    if not isinstance(text, str):
        return ''
    words = text.split()
    clean_words = [w for w in words if is_valid_vietnamese(w)]
    return ' '.join(clean_words)

In [106]:
# Áp dụng lọc
df['description_clean'] = df['description'].apply(filter_vietnamese_words,clean_description)


print(df[[ 'description_clean']])

<ipython-input-106-b195b02ef7bc>:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['description_clean'] = df['description'].apply(filter_vietnamese_words,clean_description)


In [107]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 2 cột, tên và mô tả và chỉ lấy 100 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['product_name'].fillna('') + ' ' + \
                df['sub_category'].fillna('') + ' ' + \
                df['description_clean'].fillna('').apply(lambda x: ' '.join(x.split()[:200]))


In [108]:
df.Content=df.Content.str.lower()

In [109]:
df['Content']=df['Content'].apply(filter_vietnamese_words,clean_description)
print(df['Content'])

<ipython-input-109-10cadc6e72a9>:1: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['Content']=df['Content'].apply(filter_vietnamese_words,clean_description)


0                                  áo ba lỗ thun gân body tôn dáng áo ba lỗ danh mục shopee thời trang nam áo ba lỗ chiều dài tay áo khác phong cách thể cơ đường nhiệt đới tall fit có xuất xứ việt nam dáng kiểu áo ôm sát chất liệu cotton mẫu sọc tăm có gân nổi kho hàng 75317 gửi từ bà rịa vũng tàu best tank tops for men 2021 áo ba lỗ quốc dân cho nam bảng size size m size l 55kg size xl 70kg size xxl 75kg 80kg size này là chuẩn mặc kiểu body nha quý khách còn nếu khách nào không thích mặc ôm thì tăng lên 1 size nhé còn muốn mặc rộng rộng thì tăng lên 2 size bảng size chỉ mang tính chất tham khảo vì có nhiều khách thể hình không được cân đối được tư vấn chính xác vui lòng inbox cho shop kích thước size dài 66cm rộng ngực size dài rộng 43cm ngực size 70cm rộng 47cm ngực size dài 72cm rộng 51cm ngực vòng nách m dài 25cm l dài 27cm dài 29cm xxl dài 31cm bảng size trên là mặc vừa cơ thể quá ôm sát cũng không bị rộng quá vừa vừa tuỳ vào sở thích mỗi người
1                                   

In [110]:
# Nếu có thêm thì thêm lấy mẫu ngẫu nhiên
df = df.sample(frac=0.1, random_state=42)

In [111]:
# chuẩn hóa Content_wt chỉ lấy word có nghĩa
!pip install pandarallel

#df["Content_wt"]=df["Content_wt"].apply(lambda x: ' '.join([word for word in x.split() if not word in stop_words]))

In [112]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=2, progress_bar=True)

df["Content_wt"] = df["Content"].parallel_apply(lambda x: word_tokenize(x, format="text"))


INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [113]:
# word_tokenize
#df["Content_wt"]=df["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [114]:
df[["Content", "Content_wt"]].head()

,Content,Content_wt
16126,bộ đồ ngủ ngắn tay in họa tiết cho cặp đôi đồ mặc nhà thu đông lụa cặp đôi cao cấp nam áo nữ đồ bộ pyjama pijama đồ ngủ đôi đồ bộ danh mục shopee thời trang nam đồ bộ chất liệu lụa kho hàng 23948 gửi từ nước ngoài chào kho sẵn sàng tôi có nhà máy riêng của chúng sản xuất tại trung đơn hàng được vận chuyển từ trung và sẽ đến nơi sau làm ơn hỗ trợ chúng mua sắm vui thời gian phục vụ khách hai đến chủ cửa subscribe cửa hàng để nhận coupon like ưu tiên giao hàng tận nơi sản phẩm mới mỗi ngày 5 sao của bạn là sự hỗ trợ tốt nhất cho cửa hàng của chúng tôi các đơn đặt hàng trước chiều sẽ được giao ngay trong ngày các đơn đặt hàng sau 4 giờ chiều sẽ được giao vào ngày hôm sau danh mục cung giao ngay cho dù ngoại không danh mục sản đồ ngủ 8018 nội dung thành phần vải 90 cho dù hàng tồn không chiều dài quần short danh mục đám đông có thể áp cặp đôi kích nữ nữ nữ nữ nam nam nam nam xxxl thời gian đưa ra thị mùa xuân năm 2020 độ dày của vải trọng mỏng gam mét loại cổ cổ tròn nguồn cung cấp độc quyền xuất khẩu,bộ_đồ ngủ ngắn tay in họa_tiết cho cặp đôi đồ mặc nhà thu đông lụa cặp đôi cao_cấp nam áo nữ đồ_bộ pyjama pijama_đồ ngủ đôi đồ bộ danh_mục shopee thời_trang nam đồ_bộ chất_liệu lụa kho hàng 23948 gửi từ nước_ngoài chào_kho sẵn_sàng tôi có nhà_máy riêng của chúng sản_xuất tại trung_đơn hàng được vận_chuyển từ trung và sẽ đến_nơi sau làm ơn hỗ_trợ chúng mua_sắm vui thời_gian phục_vụ khách hai đến chủ cửa subscribe cửa_hàng để nhận coupon like ưu_tiên giao hàng tận nơi sản_phẩm mới mỗi ngày 5 sao của bạn là sự hỗ_trợ tốt nhất cho cửa_hàng của chúng_tôi các đơn đặt_hàng trước chiều sẽ được giao ngay trong ngày các đơn đặt_hàng sau 4 giờ chiều sẽ được giao vào ngày hôm sau danh_mục cung giao ngay cho_dù ngoại không danh_mục_sản đồ ngủ 8018 nội_dung thành_phần vải 90 cho_dù hàng tồn không chiều dài quần_short danh_mục đám đông có_thể áp_cặp đôi_kích nữ nữ nữ nữ nam nam nam nam xxxl thời_gian đưa ra thị mùa xuân năm 2020 độ dày của vải_trọng mỏng gam mét loại cổ cổ_tròn nguồn cung_cấp độc_quyền xuất_khẩu
14055,áo thun hình thật tay lỡ i need mẫu cặp mới đồ bộ danh mục shopee thời trang nam đồ bộ chất liệu khác loại trang phục áo xuất xứ việt nam mẫu in chiều dài tay áo tay lỡ kho hàng 40 gửi từ hồ chí minh áo_thun_unisex_form_rộng _giấu_quần áo thun tay lỡ unisex hot nhất thị bao đẹp bao độc bao chất thun cotton co dãn 4 vải vải vải thoáng không xù mực in nhập không bông sắc không phai không gây hại cho tốt cho sức khỏe hàng loại 1 chuẩn from unisex chất lừ nhé quý áo tay lỡ free size from đại chuẩn tối đa 68kg và thích dạng ôm còn thích rộng unisex thì từ 55 đến 58kg thích siêu rộng và dài nhiều tầm 42kg đến 52kg xin liên hệ nhân viên tư vấn để dc tư vấn tốt kế trẻ năng động rất phù hợp tuổi size 68kg cam thật giống hình hình sau áo bảo vải chất lượng hàng đẹp xứng đáng đồng tiền bát được kiểm tra kĩ cẩn thận và tư vấn nhiệt tình trước khi gói hàng giao cho quý tiền nếu hàng lỗi không giống sai quý khách cứ an tâm mua sắm tại shop thương hiệu mạc áo,áo thun_hình thật tay lỡ i need mẫu cặp mới đồ_bộ danh_mục shopee thời_trang nam đồ_bộ chất_liệu khác loại trang_phục áo xuất_xứ việt_nam mẫu in chiều dài tay_áo tay lỡ kho hàng 40 gửi từ hồ chí_minh_áo_thun_unisex_form_rộng _giấu_quần áo thun tay lỡ unisex_hot nhất_thị bao đẹp bao_độc bao_chất thun_cotton co_dãn 4 vải vải vải thoáng không xù_mực in nhập không bông_sắc không phai không gây hại cho tốt cho sức khỏe hàng loại 1 chuẩn from_unisex chất_lừ nhé quý áo tay lỡ free size from đại_chuẩn tối_đa 68 kg và thích dạng ôm còn thích rộng unisex thì từ 55 đến 58 kg thích siêu_rộng và dài nhiều tầm 42 kg đến 52 kg xin liên_hệ nhân_viên tư_vấn để dc tư_vấn tốt_kế trẻ năng_động rất phù_hợp tuổi size 68 kg cam thật giống hình_hình sau áo bảo vải chất_lượng hàng đẹp xứng_đáng đồng_tiền bát được kiểm_tra kĩ_cẩn_thận và tư_vấn nhiệt_tình trước khi gói hàng giao cho quý tiền nếu hàng lỗi không giống sai quý_khách cứ an_tâm mua_sắm tại shop thương_hiệu_mạc áo
30344,móc khóa cute

### Gemsim


In [115]:
# Tokenize(split) the sentences into words
content_gem = [[text for text in x.split()] for x in df.Content_wt]

In [116]:
len(content_gem)

2113

In [117]:
content_gem[:1]

[['bộ_đồ',
  'ngủ',
  'ngắn',
  'tay',
  'in',
  'họa_tiết',
  'cho',
  'cặp',
  'đôi',
  'đồ',
  'mặc',
  'nhà',
  'thu',
  'đông',
  'lụa',
  'cặp',
  'đôi',
  'cao_cấp',
  'nam',
  'áo',
  'nữ',
  'đồ_bộ',
  'pyjama',
  'pijama_đồ',
  'ngủ',
  'đôi',
  'đồ',
  'bộ',
  'danh_mục',
  'shopee',
  'thời_trang',
  'nam',
  'đồ_bộ',
  'chất_liệu',
  'lụa',
  'kho',
  'hàng',
  '23948',
  'gửi',
  'từ',
  'nước_ngoài',
  'chào_kho',
  'sẵn_sàng',
  'tôi',
  'có',
  'nhà_máy',
  'riêng',
  'của',
  'chúng',
  'sản_xuất',
  'tại',
  'trung_đơn',
  'hàng',
  'được',
  'vận_chuyển',
  'từ',
  'trung',
  'và',
  'sẽ',
  'đến_nơi',
  'sau',
  'làm',
  'ơn',
  'hỗ_trợ',
  'chúng',
  'mua_sắm',
  'vui',
  'thời_gian',
  'phục_vụ',
  'khách',
  'hai',
  'đến',
  'chủ',
  'cửa',
  'subscribe',
  'cửa_hàng',
  'để',
  'nhận',
  'coupon',
  'like',
  'ưu_tiên',
  'giao',
  'hàng',
  'tận',
  'nơi',
  'sản_phẩm',
  'mới',
  'mỗi',
  'ngày',
  '5',
  'sao',
  'của',
  'bạn',
  'là',
  'sự',
  'hỗ_trợ',


In [118]:
# Tiền xử lý dữ liệu
content_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in content_gem] # xem xét có cần bỏ các con số hay không
content_gem_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/', "'", '&']] for text in  content_gem_re] # kiểm tra nội dung và đưa vào các ký tự đặc biệt
content_gem_re = [[t for t in text if not t in stop_words] for text in content_gem_re] # stopword
# Thực hiện các tiền xử lý khác => Đã tiền xử lý ở trên rồi
# Nên viết phần tiền xử lý này vào function để có thể tái sử dụng khi cần

In [119]:
content_gem_re[:1]


[['bộ_đồ',
  'ngắn',
  'in',
  'họa_tiết',
  'cặp',
  'đôi',
  'đồ',
  'mặc',
  'thu',
  'đông',
  'lụa',
  'cặp',
  'đôi',
  'cao_cấp',
  'nam',
  'áo',
  'nữ',
  'đồ_bộ',
  'pyjama',
  'pijama_đồ',
  'đôi',
  'đồ',
  'danh_mục',
  'shopee',
  'thời_trang',
  'nam',
  'đồ_bộ',
  'chất_liệu',
  'lụa',
  'kho',
  'hàng',
  'gửi',
  'nước_ngoài',
  'chào_kho',
  'sẵn_sàng',
  'nhà_máy',
  'sản_xuất',
  'trung_đơn',
  'hàng',
  'vận_chuyển',
  'trung',
  'ơn',
  'mua_sắm',
  'vui',
  'phục_vụ',
  'hai',
  'chủ',
  'cửa',
  'subscribe',
  'cửa_hàng',
  'coupon',
  'like',
  'ưu_tiên',
  'giao',
  'hàng',
  'tận',
  'sản_phẩm',
  'tốt',
  'cửa_hàng',
  'đơn',
  'đặt_hàng',
  'chiều',
  'giao',
  'đơn',
  'đặt_hàng',
  'chiều',
  'giao',
  'hôm',
  'danh_mục',
  'cung',
  'giao',
  'cho_dù',
  'ngoại',
  'không',
  'danh_mục_sản',
  'đồ',
  'nội_dung',
  'thành_phần',
  'vải',
  'cho_dù',
  'hàng',
  'tồn',
  'không',
  'chiều',
  'quần_short',
  'danh_mục',
  'đám',
  'đông',
  'áp_cặp',
  

In [120]:
# Obtain the number of features based on dictionary: Use corpora.Dictionary
dictionary = corpora.Dictionary(content_gem_re)

In [121]:
# List of features in dictionary
dictionary.token2id

{'bộ_đồ': 0,
 'cao_cấp': 1,
 'chiều': 2,
 'cho_dù': 3,
 'chào_kho': 4,
 'chất_liệu': 5,
 'chủ': 6,
 'coupon': 7,
 'cung': 8,
 'cặp': 9,
 'cổ': 10,
 'cổ_tròn': 11,
 'cửa': 12,
 'cửa_hàng': 13,
 'danh_mục': 14,
 'danh_mục_sản': 15,
 'dày': 16,
 'gam': 17,
 'giao': 18,
 'gửi': 19,
 'hai': 20,
 'hàng': 21,
 'hôm': 22,
 'họa_tiết': 23,
 'in': 24,
 'kho': 25,
 'không': 26,
 'like': 27,
 'lụa': 28,
 'mua_sắm': 29,
 'mét': 30,
 'mùa': 31,
 'mặc': 32,
 'mỏng': 33,
 'nam': 34,
 'ngoại': 35,
 'ngắn': 36,
 'nhà_máy': 37,
 'nước_ngoài': 38,
 'nội_dung': 39,
 'nữ': 40,
 'phục_vụ': 41,
 'pijama_đồ': 42,
 'pyjama': 43,
 'quần_short': 44,
 'shopee': 45,
 'subscribe': 46,
 'sản_phẩm': 47,
 'sản_xuất': 48,
 'sẵn_sàng': 49,
 'thu': 50,
 'thành_phần': 51,
 'thị': 52,
 'thời_trang': 53,
 'trung': 54,
 'trung_đơn': 55,
 'tận': 56,
 'tốt': 57,
 'tồn': 58,
 'vui': 59,
 'vải': 60,
 'vải_trọng': 61,
 'vận_chuyển': 62,
 'xuân': 63,
 'xuất_khẩu': 64,
 'xxxl': 65,
 'áo': 66,
 'áp_cặp': 67,
 'đám': 68,
 'đôi': 69,
 

In [122]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)

In [123]:
feature_cnt

13755

In [124]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in content_gem_re]

In [125]:
corpus[0] # id, so lan xuat hien cua token trong van ban/ san pham

[(0, 1),
 (1, 1),
 (2, 3),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 3),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 4),
 (19, 1),
 (20, 1),
 (21, 4),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 6),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 5),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 3),
 (70, 1),
 (71, 2),
 (72, 2),
 (73, 2),
 (74, 3),
 (75, 2),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1)]

In [126]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                            num_features = feature_cnt)
# ma tran: n x n

In [127]:
df_1 = pd.DataFrame(index)
df_1

,0,1,2,3,4,5,6,7,8,9,...,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112
0,1.000000,0.044898,0.016512,0.042245,0.016251,0.048944,0.069899,0.047667,0.056677,0.014720,...,0.045083,0.028340,0.012949,0.075577,0.012534,0.010846,0.028342,0.009859,0.074829,0.005085
1,0.044898,1.000000,0.010228,0.026332,0.016299,0.238946,0.069029,0.061524,0.031064,0.098109,...,0.058912,0.035359,0.005935,0.036179,0.013404,0.020436,0.011027,0.050060,0.042345,0.029208
2,0.016512,0.010228,1.000000,0.003982,0.006389,0.012613,0.012164,0.003247,0.026447,0.013681,...,0.040213,0.016313,0.006705,0.009016,0.017773,0.010995,0.003867,0.000815,0.006346,0.008849
3,0.042245,0.026332,0.003982,1.000000,0.011441,0.006958,0.021040,0.018615,0.007761,0.028966,...,0.013148,0.056460,0.007017,0.001454,0.004485,0.010237,0.003288,0.001735,0.032434,0.026210
4,0.016251,0.016299,0.006389,0.011441,1.000000,0.014250,0.004523,0.013752,0.014205,0.015761,...,0.024252,0.030898,0.017819,0.018744,0.024011,0.044341,0.017425,0.025047,0.004604,0.018218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108,0.010846,0.020436,0.010995,0.010237,0.044341,0.011788,0.009344,0.014849,0.019825,0.017441,...,0.019500,0.029295,0.006376,0.018952,0.018990,1.000000,0.026052,0.010038,0.005479,0.019393
2109,0.028342,0.011027,0.003867,0.003288,0.017425,0.001048,0.003144,0.014372,0.023751,0.005364,...,0.024286,0.010716,0.006571,0.032474,0.000584,0.026052,1.000000,0.023185,0.000026,0.007222
2110,0.009859,0.050060,0.000815,0.001735,0.025047,0.036300,0.024445,0.008754,0.071620,0.017488,...,0.059476,0.027449,0.000060,0.084742,0.011890,0.010038,0.023185,1.000000,0.000040,0.013883
2111,0.074829,0.042345,0.006346,0.032434,0.004604,0.011551,0.105895,0.028119,0.013482,0.042374,...,0.030834,0.005117,0.010688,0.046505,0.000033,0.005479,0.000026,0.000040,1.000000,0.022634


In [128]:
# Khi khách chọn 1 sản phẩm để xem thông tin
# Giả sử là chọn khách sạn đầu tiên với product_id = 19656
# HV cần xử lý chi tiết phần này
# Ở đây tạm lấy phần content đã được tiền xử lý
content_gem_re[:1]

[['bộ_đồ',
  'ngắn',
  'in',
  'họa_tiết',
  'cặp',
  'đôi',
  'đồ',
  'mặc',
  'thu',
  'đông',
  'lụa',
  'cặp',
  'đôi',
  'cao_cấp',
  'nam',
  'áo',
  'nữ',
  'đồ_bộ',
  'pyjama',
  'pijama_đồ',
  'đôi',
  'đồ',
  'danh_mục',
  'shopee',
  'thời_trang',
  'nam',
  'đồ_bộ',
  'chất_liệu',
  'lụa',
  'kho',
  'hàng',
  'gửi',
  'nước_ngoài',
  'chào_kho',
  'sẵn_sàng',
  'nhà_máy',
  'sản_xuất',
  'trung_đơn',
  'hàng',
  'vận_chuyển',
  'trung',
  'ơn',
  'mua_sắm',
  'vui',
  'phục_vụ',
  'hai',
  'chủ',
  'cửa',
  'subscribe',
  'cửa_hàng',
  'coupon',
  'like',
  'ưu_tiên',
  'giao',
  'hàng',
  'tận',
  'sản_phẩm',
  'tốt',
  'cửa_hàng',
  'đơn',
  'đặt_hàng',
  'chiều',
  'giao',
  'đơn',
  'đặt_hàng',
  'chiều',
  'giao',
  'hôm',
  'danh_mục',
  'cung',
  'giao',
  'cho_dù',
  'ngoại',
  'không',
  'danh_mục_sản',
  'đồ',
  'nội_dung',
  'thành_phần',
  'vải',
  'cho_dù',
  'hàng',
  'tồn',
  'không',
  'chiều',
  'quần_short',
  'danh_mục',
  'đám',
  'đông',
  'áp_cặp',
  

In [129]:
view_content = content_gem_re[:1][0]

In [130]:
# Convert search words into Sparse Vectors
kw_vector = dictionary.doc2bow(view_content)

In [131]:
kw_vector

[(0, 1),
 (1, 1),
 (2, 3),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 3),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 4),
 (19, 1),
 (20, 1),
 (21, 4),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 6),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 5),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 3),
 (70, 1),
 (71, 2),
 (72, 2),
 (73, 2),
 (74, 3),
 (75, 2),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1)]

In [132]:
# similarity calculation
sim = index[tfidf[kw_vector]]

In [133]:
# Ở đây đang in tuần tự, nên chỉnh lại in từ item có value cao -> thấp
for i in range(len(sim)):
    # Vì lấy mẫu đầu tiên để xem nên bỏ qua mẫu đầu tiên, cần viết lại function này để linh hoạt lấy ra các sp liên quan
    if i!=0:
        print('keyword is similar to doc_index %d: %.4f' % (i, sim[i]))

keyword is similar to doc_index 1: 0.0449
keyword is similar to doc_index 2: 0.0165
keyword is similar to doc_index 3: 0.0422
keyword is similar to doc_index 4: 0.0163
keyword is similar to doc_index 5: 0.0489
keyword is similar to doc_index 6: 0.0699
keyword is similar to doc_index 7: 0.0477
keyword is similar to doc_index 8: 0.0567
keyword is similar to doc_index 9: 0.0147
keyword is similar to doc_index 10: 0.0269
keyword is similar to doc_index 11: 0.0232
keyword is similar to doc_index 12: 0.0378
keyword is similar to doc_index 13: 0.0395
keyword is similar to doc_index 14: 0.0054
keyword is similar to doc_index 15: 0.0696
keyword is similar to doc_index 16: 0.0578
keyword is similar to doc_index 17: 0.0130
keyword is similar to doc_index 18: 0.0516
keyword is similar to doc_index 19: 0.0201
keyword is similar to doc_index 20: 0.0050
keyword is similar to doc_index 21: 0.0754
keyword is similar to doc_index 22: 0.0368
keyword is similar to doc_index 23: 0.0052
keyword is similar t

In [134]:
# Giả sử 3 sp liên quan nhất với sp có index đầu tiên
content_famillier = df.iloc[[9, 8, 6]]

In [135]:
content_famillier

,product_id,product_name,category,sub_category,link,image,price,rating,description,description_clean,Content,Content_wt
19332,24298,IRON MAN COMICS ORIGINAL ( áo đen logo chính giữa ),Thời Trang Nam,Đồ Hóa Trang,https://shopee.vn/IRON-MAN-COMICS-ORIGINAL-(-%C3%A1o-%C4%91en-logo-ch%C3%ADnh-gi%E1%BB%AFa-)-i.12144284.1188085872?sp_atk=ced249dc-9d19-4b5d-ad6e-7b1a5878d122&xptdk=ced249dc-9d19-4b5d-ad6e-7b1a5878d122,https://cf.shopee.vn/file/ac0521b12871c2447420de86e569b873_tn,249000.0,5.0,"Danh Mục\nShopee\nThời Trang Nam\nĐồ Hóa Trang\nMẫu\nIn\nKho hàng\n2\nGửi từ\nTP. Hồ Chí Minh\n\nÁO THUN THÁI - MARVEL - DC\nMade in thailand\nUnisex nam nữ đều mặc đc \n#IRONMAN\nSIZE M : NGANG 51 DÀI 67 ( < 55KG )\nSIZE L : NGANG 55 DÀI 70 ( < 67KG)\nSIZE XL : NGANG 56 DÀI 74 (< 75KG )\nHình in đẹp , giặt không xuống màu , thun mát thoải mái \nGiá : #250k\nMua 3 giảm 10k/áo\nMua 6 giảm 20k/áo\nMua 12 giảm 30k/áo ( Bất kì áo thun nào )\nSize và màu như hình \nXem thêm tại : #az250ld\n🚀SHIP HÀNG TOÀN QUỐC\n🔥Free ship chỉ từ 200k qua Shopee , hỗ trợ tối đa 40k ship\n🛒Mua online inbox Page ( Facebook : Littledevil.shop )\n✅Phone : 090 6666 838\n🌈Facebook : Littledevil.shop\n⭐️Shopee : littledevilshop",Danh Mục Shopee Thời Trang Nam Đồ Hóa Trang Mẫu In Kho hàng 2 Gửi từ Hồ Chí Minh ÁO THUN THÁI MARVEL DC Made in thailand Unisex nam nữ đều mặc đc SIZE M NGANG 51 DÀI 67 55KG SIZE L NGANG 55 DÀI 70 SIZE XL NGANG 56 DÀI 74 75KG Hình in đẹp giặt không xuống màu thun mát thoải mái Giá Mua 3 giảm Mua 6 giảm Mua 12 giảm Bất kì áo thun nào Size và màu như hình Xem thêm tại HÀNG TOÀN QUỐC ship chỉ từ 200k qua Shopee hỗ trợ tối đa 40k ship online inbox Page Facebook 090 6666 838 littledevilshop,iron man comics original áo đen logo chính giữa đồ hóa trang danh mục shopee thời trang nam đồ hóa trang mẫu in kho hàng 2 gửi từ hồ chí minh áo thun thái marvel dc made in thailand unisex nam nữ đều mặc đc size m ngang 51 dài 67 55kg size l ngang 55 dài 70 size xl ngang 56 dài 74 75kg hình in đẹp giặt không xuống màu thun mát thoải mái giá mua 3 giảm mua 6 giảm mua 12 giảm bất kì áo thun nào size và màu như hình xem thêm tại hàng toàn quốc ship chỉ từ 200k qua shopee hỗ trợ tối đa 40k ship online inbox page facebook 090 6666 838 littledevilshop,iron_man comics original áo đen logo chính giữa đồ hóa_trang danh_mục shopee thời_trang nam_đồ hóa_trang mẫu in kho hàng 2 gửi từ hồ chí_minh áo thun_thái marvel dc made in thailand unisex nam_nữ đều mặc đc size m ngang 51 dài 67 55 kg size l ngang 55 dài 70 size xl ngang 56 dài 74 75 kg hình in đẹp giặt không xuống màu thun mát thoải_mái giá mua 3 giảm mua 6 giảm mua 12 giảm bất_kì áo_thun nào size và màu như hình xem thêm tại hàng toàn_quốc ship chỉ từ 200 k qua shopee hỗ_trợ tối_đa 40 k ship online inbox page facebook 090 6666 838 littledevilshop
15831,233497,Bộ thể thao nam bigsize cao cấp đồ bộ nam cổ bẻ ngắn tay xuất khẩu BTT 003,Thời Trang Nam,Đồ Bộ,https://shopee.vn/B%E1%BB%99-th%E1%BB%83-thao-nam-bigsize-cao-c%E1%BA%A5p-%C4%91%E1%BB%93-b%E1%BB%99-nam-c%E1%BB%95-b%E1%BA%BB-ng%E1%BA%AFn-tay-xu%E1%BA%A5t-kh%E1%BA%A9u-BTT-003-i.148211991.5829604381?sp_atk=076646ab-3564-4c2a-89f4-095da1a5eb42&xptdk=076646ab-3564-4c2a-89f4-095da1a5eb42,https://cf.shopee.vn/file/c0fdc2fba13957adb90f5a106f517c1a_tn,459000.0,5.0,"Danh Mục\nShopee\nThời Trang Nam\nĐồ Bộ\nĐịa chỉ tổ chức chịu trách nhiệm sản xuất\nĐang cập nhật\nMùa\nMùa hè\nPhong cách\nCơ bản, Nhiệt đới, Thể thao\nTên tổ chức chịu trách nhiệm sản xuất\nĐang cập nhật\nCổ áo\nCổ bẻ\nKho hàng\n12339\nGửi từ\nQuận Hồng Bàng, Hải Phòng\n\nBộ thể thao nam bigsize cao cấp đồ bộ nam cổ bẻ ngắn tay xuất khẩu BTT 003\n\nBộ thể thao nam với thiết kế mới nhất trong năm nay \nBộ thể thao nam cổ bẻ chất liệu vải DRYFIT co dãn 4 chiều cao cấp chính hãng, co dãn cực tốt, làm mát nhanh chóng\nBộ quần áo thể thao nam với cổ dệt rất tinh tế \nĐồ bộ nam với quần thun siêu co dãn, các túi đều có khóa kéo \nBộ đồ thể thao nam với phối màu tinh tế sang trọng nhưng vẫn không kém tính thể thao\nBộ thể th

In [136]:
def recommend_similar_products(df, index_matrix, tfidf_model, dictionary, content_list, product_index=0, top_n=3):
    """
    Trả về top N sản phẩm tương tự với sản phẩm đang được xem.
    """
    # Bước 1: Lấy nội dung sản phẩm gốc (đã tiền xử lý)
    query_doc = content_list[product_index]
    query_vec = dictionary.doc2bow(query_doc)

    # Bước 2: Tính điểm tương đồng
    sims = index_matrix[tfidf_model[query_vec]]

    # Bước 3: Lấy top N sản phẩm có điểm cao nhất (bỏ qua chính nó)
    sim_scores = list(enumerate(sims))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Bỏ sản phẩm đang xem (index = product_index)
    top_similar_idx = [i for i, score in sim_scores if i != product_index][:top_n]
    top_similar_scores = [score for i, score in sim_scores if i != product_index][:top_n]

    # Bước 4: Tạo bảng kết quả
    result = df.iloc[top_similar_idx].copy()
    result["similarity_score"] = top_similar_scores

    return result


In [137]:
for i in range(5):  # Gợi ý cho 3 sản phẩm đầu tiên
    result = recommend_similar_products(df, index, tfidf, dictionary, content_gem_re, product_index=i, top_n=3)
    result.to_excel(f"recommend_product_{i}.xlsx", index=False)


In [138]:
print(result)


       product_id  \
45317       17605   
47628      173033   
47599      173004   

                                                                                     product_name  \
45317                                Combo 20 Đôi Tất Nam Cổ Dài Uni Nhật Bản Giá Rẻ | Vớ Nam Uni   
47628  Tất Nam Trịnh Trung Chính Hãng- Tất Bàn Chân Chất Liệu Cotton Không Bai,Xù Siêu Đẹp(1 Đôi)   
47599                                              [được chọn màu] tất nam cổ ngắn hàng xuất Nhật   

             category sub_category  \
45317  Thời Trang Nam       Vớ/Tất   
47628  Thời Trang Nam       Vớ/Tất   
47599  Thời Trang Nam       Vớ/Tất   

                                                                                                                                                                                                                                                                                                                               link  \
45317                         

# Cosine_similarity


In [139]:
from numpy import dot
from numpy.linalg import norm
A = [1,2]
B = [2,2]
cos_sim = dot(A,B) / (norm(A)*norm(B))
print(cos_sim)

0.9486832980505138


In [140]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [141]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112
0,1.000000,0.081058,0.032816,0.072261,0.039595,0.080005,0.098040,0.087767,0.104213,0.031351,...,0.087292,0.062314,0.036092,0.121473,0.036658,0.032842,0.052594,0.034342,0.104829,0.028323
1,0.081058,1.000000,0.027453,0.046972,0.043370,0.286514,0.098648,0.124675,0.057666,0.155639,...,0.134202,0.078214,0.019437,0.062554,0.022905,0.043777,0.023454,0.080793,0.092303,0.053667
2,0.032816,0.027453,1.000000,0.013678,0.020102,0.024033,0.026294,0.013107,0.041408,0.030551,...,0.059920,0.036080,0.018534,0.031456,0.030046,0.029246,0.013496,0.009116,0.024226,0.022274
3,0.072261,0.046972,0.013678,1.000000,0.026218,0.021959,0.036500,0.040992,0.038856,0.032993,...,0.027856,0.082760,0.021869,0.014379,0.025685,0.025657,0.013292,0.015392,0.046306,0.058467
4,0.039595,0.043370,0.020102,0.026218,1.000000,0.032655,0.018972,0.031997,0.039913,0.033998,...,0.058086,0.056187,0.034348,0.042435,0.045853,0.074771,0.031327,0.047481,0.014787,0.039632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108,0.032842,0.043777,0.029246,0.025657,0.074771,0.032689,0.029559,0.027238,0.045061,0.037218,...,0.055174,0.061904,0.019633,0.040519,0.038764,1.000000,0.042920,0.028057,0.021815,0.042498
2109,0.052594,0.023454,0.013496,0.013292,0.031327,0.009848,0.013575,0.026989,0.055210,0.014213,...,0.042292,0.027600,0.019835,0.057943,0.011672,0.042920,1.000000,0.043242,0.011012,0.020690
2110,0.034342,0.080793,0.009116,0.015392,0.047481,0.051626,0.044640,0.025969,0.134106,0.024222,...,0.086835,0.058691,0.009755,0.118700,0.030899,0.028057,0.043242,1.000000,0.012255,0.027303
2111,0.104829,0.092303,0.024226,0.046306,0.014787,0.036584,0.114915,0.062455,0.026824,0.093154,...,0.078474,0.042654,0.028461,0.059780,0.011605,0.021815,0.011012,0.012255,1.000000,0.043586


In [142]:
# # Hàm đề xuất khách sạn
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
# def get_recommendations(id, cosine_sim=cosine_sim, nums=5):
#     # Check if the product_id exists in the DataFrame
#     if id not in df['product_id'].values:
#         print(f" Product ID {id} not found. Dùng thử: {df['product_id'].iloc[0]}")
#         return []

#     idx = df.index[df['product_id'] == id][0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:nums+1]  # Lấy nums sp tương tự nhất
#     product_indices = [i[0] for i in sim_scores]
#     return df['product_name'].iloc[product_indices]

In [143]:
# # Gọi function
# recommendations = get_recommendations(4,cosine_sim,3)
# print(recommendations)

In [144]:
# recommendations = get_recommendations(233792,cosine_sim,3)
# print(recommendations)

In [145]:
# Hàm đề xuất
# với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations(id, cosine_sim=cosine_sim, nums=5):
    # Check if the product_id exists in the DataFrame
    if id not in df['product_id'].values:
        print(f" Product ID {id} not found. Dùng thử: {df['product_id'].iloc[0]}")
        return []
    # Get the index of the product that matches the id
    idx = df[df['product_id'] == id].index[0]
    # Ensure idx is within the bounds of cosine_sim
    idx = min(idx, cosine_sim.shape[0] - 1)  # Limit idx to the maximum row index

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy nums sp tương tự nhất
    product_indices = [i[0] for i in sim_scores]

    #Ensure product_indices are within the bounds of df
    product_indices = [i for i in product_indices if i < len(df)]

    return df['product_name'].iloc[product_indices]

In [146]:
recommendations = get_recommendations(233792,cosine_sim,3)
print(recommendations)

23749                                                                         Hộp 4 Sịp Đùi Boxer Thông Hơi Cao Cấp Dành Cho Nam
22866                                                                          Quần Lót Nam💞FREE SHIP💞Quần Sịp Đùi Nam Thông Hơi
22565    Quần Lót Nam Thoáng Mát Nam Tính Quần Boxer Cotton Thấm Hút Mồ Hôi Quần Sịp Nam Thun Lạnh Co Giãn 4 Chiều Cao Cấp GALIO
Name: product_name, dtype: object


In [147]:
# prompt: kết xuất với id 233792 là sản phẩm gì trước khi tìm những sản phẩm liên quan

# Assuming 'df' is your DataFrame and 'cosine_sim' is your similarity matrix
# Get the product name for product_id 233792
product_name = df[df['product_id'] == 233792]['product_name'].iloc[0]
print(f"Sản phẩm với ID 233792 là: {product_name}")


Sản phẩm với ID 233792 là: Bộ Đồ Ngủ Ngắn Tay In Họa Tiết Cho Cặp Đôi đồ mặc nhà thu đông lụa cặp đôi cao cấp nam áo nữ đồ bộ pyjama pijama Đồ ngủ đôi


In [148]:
# SO SÁNH KẾT QUẢ



In [149]:
# sample_index = 0
# sample_content = df['Content'].iloc[sample_index]


In [150]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# tfidf = TfidfVectorizer()
# tfidf_matrix = tfidf.fit_transform(df['Content'])

# cos_sim_scores = cosine_similarity(tfidf_matrix[sample_index], tfidf_matrix).flatten()
# cos_top_indices = cos_sim_scores.argsort()[-6:-1][::-1]  # bỏ chính nó


In [151]:
# from gensim import corpora, models, similarities

# # Tokenize content
# tokenized_content = [text.split() for text in df['Content']]

# # Dictionary + Corpus
# dictionary = corpora.Dictionary(tokenized_content)
# corpus = [dictionary.doc2bow(text) for text in tokenized_content]

# # TF-IDF model + similarity index
# tfidf_model = models.TfidfModel(corpus)
# index = similarities.SparseMatrixSimilarity(tfidf_model[corpus], num_features=len(dictionary))

# # Query vector
# query_vec = dictionary.doc2bow(tokenized_content[sample_index])
# gensim_scores = index[tfidf_model[query_vec]]
# gensim_top_indices = gensim_scores.argsort()[-6:-1][::-1]


In [152]:
# import pandas as pd

# compare_df = pd.DataFrame({
#     "Cosine Top Products": df.iloc[cos_top_indices]["product_name"].values,
#     "Gensim Top Products": df.iloc[gensim_top_indices]["product_name"].values
# })
# display(compare_df)


,Cosine Top Products,Gensim Top Products
0,Vớ kiểu dáng đơn giản thấm hút mồ hôi tốt thời trang cho nam,Vớ kiểu dáng đơn giản thấm hút mồ hôi tốt thời trang cho nam
1,Đôi Tất Cotton Dài Họa Tiết Sọc Retro Thời Trang Cho Nam Nữ,Đôi Tất Cotton Dài Họa Tiết Sọc Retro Thời Trang Cho Nam Nữ
2,Bộ Đồ Ngủ 2 Món Xinh Xắn Thời Trang Cho Cặp Đôi,Bộ Đồ Ngủ 2 Món Xinh Xắn Thời Trang Cho Cặp Đôi
3,Đôi tất cổ ngắn màu trơn 3 màu tùy chọn thời trang cho nam nữ,Áo kimono hóa trang nhân vật hoạt hình tokyo revengers mikey draken độc đáo
4,Áo kimono hóa trang nhân vật hoạt hình tokyo revengers mikey draken độc đáo,Set 5 đôi vớ ngắn khử mùi chống hôi chân thoải mái đơn giản


In [153]:
# Save cosine_sim to file
import pickle
with open('products_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('products_cosine_sim.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [157]:
# def get_recommendations(df, ma_san_pham, cosine_sim=cosine_sim, nums=5):
#     # Get the index of the hotel that matches the hotel_id
#     matching_indices = df.index[df['product_id'] == ma_san_pham].tolist()
#     if not matching_indices:
#         print(f"No hotel found with ID: {ma_san_pham}")
#         return pd.DataFrame()  # Return an empty DataFrame if no match
#     idx = matching_indices[0]

#     # Get the pairwise similarity scores of all hotels with that hotel
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the hotels based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar hotels (Ignoring the hotel itself)
#     sim_scores = sim_scores[1:nums+1]

#     # Get the hotel indices
#     product_indices = [i[0] for i in sim_scores]

#     # Return the top n most similar hotels as a DataFrame
#     return df.iloc[product_indices]

In [160]:
def get_recommendations(df, ma_san_pham, cosine_sim=cosine_sim, nums=5):
    # Get the index of the product that matches the ma_san_pham
    # and ensure it is within the bounds of the cosine_sim matrix
    matching_indices = df.index[df['product_id'] == ma_san_pham].tolist()
    if not matching_indices:
        print(f"No product found with ID: {ma_san_pham}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]
    idx = min(idx, cosine_sim.shape[0] - 1)  # Limit idx to the maximum row index

    # Get the pairwise similarity scores of all products with the target product
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the products based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top similar products (ignoring the product itself)
    sim_scores = sim_scores[1:nums + 1]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top similar products as a DataFrame
    return df.iloc[product_indices]

In [161]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df,233792, cosine_sim=cosine_sim_new, nums=3)
recommendations

,product_id,product_name,category,sub_category,link,image,price,rating,description,description_clean,Content,Content_wt
23749,251410,Hộp 4 Sịp Đùi Boxer Thông Hơi Cao Cấp Dành Cho Nam,Thời Trang Nam,Đồ Lót,https://shopee.vn/H%E1%BB%99p-4-S%E1%BB%8Bp-%C4%90%C3%B9i-Boxer-Th%C3%B4ng-H%C6%A1i-Cao-C%E1%BA%A5p-D%C3%A0nh-Cho-Nam-i.3053599.2324432323?sp_atk=a23f686c-ef94-4d62-affc-86acf8f41f39,https://cf.shopee.vn/file/2aba52c1d245259b737d88e3a1c10032_tn,99000.0,5.0,"Danh Mục\nShopee\nThời Trang Nam\nĐồ Lót\nQuần Lót\nKiểu quần lót\nBoxers & Trunks\nKho hàng\n813\nGửi từ\nHuyện Hoài Đức, Hà Nội\n\nHỘP 4 QUẦN LÓT NAM BOXER ĐÙI THÔNG HƠI CAO CẤP\nQuần sịp đùi nam dáng dài, ôm sát body trẻ trung, cuốn hút, tạo cảm giác vô cùng thoải mái, tự tin vận động , tạo vẻ quyến rũ đầy nam tính\n Quần có 3 size:\n- Size L(50-56kg),\n-Size XL(57-66kg),\n-Size XXL (67-72kg)\nChất liệu : cotton, ployeste,spander\n- Màu sắc : có 3 màu : đen , xanh, tím\n- Hộp 4 chiếc \nĐẶC TÍNH NỔI BẬT\n- Quần sịp đùi nam dáng dài đến đùi, ôm sát body trẻ trung, cuốn hút mà vẫn tạo cảm giác thoải mái.\n- Thiết kế đặc biệt cho bạn tự tin vận động, tạo vẻ quyến rũ đầy nam tính\n- Cạp chun bản rộng co giãn, phù hợp với nhiều lứa tuổi, không gây hằn trên da\n- Khử mùi nhanh, co giãn 4 chiều tạo cảm giác thoải mái không bí\nCạp chun bản rộng co giãn, thoải mái không hằn trên da\n-Quần siêu Thoáng mát cho anh em ngày hè nóng nực\n#quanlotnam #sipduinam #sipduithonghoinam #sipthonghoi #sipthonghoinam #boxer #boxerdui #hop5chiec",Danh Mục Shopee Thời Trang Nam Đồ Lót Quần Lót Kiểu quần lót Boxers Trunks Kho hàng 813 Gửi từ Huyện Hoài Hà Nội HỘP 4 QUẦN LÓT NAM BOXER ĐÙI THÔNG HƠI CAO CẤP Quần sịp đùi nam dáng ôm sát body trẻ cuốn tạo cảm giác vô cùng thoải tự tin vận động tạo vẻ quyến rũ đầy nam tính Quần có 3 Size XXL Chất liệu Màu sắc có 3 màu đen tím Hộp 4 chiếc ĐẶC TÍNH NỔI BẬT Quần sịp đùi nam dáng dài đến ôm sát body trẻ cuốn hút mà vẫn tạo cảm giác thoải Thiết kế đặc biệt cho bạn tự tin vận tạo vẻ quyến rũ đầy nam tính Cạp chun bản rộng co phù hợp với nhiều lứa không gây hằn trên da Khử mùi co giãn 4 chiều tạo cảm giác thoải mái không bí Cạp chun bản rộng co thoải mái không hằn trên da siêu Thoáng mát cho anh em ngày hè nóng nực,hộp 4 sịp đùi boxer thông hơi cao cấp dành cho nam đồ lót danh mục shopee thời trang nam đồ lót quần lót kiểu quần lót boxers trunks kho hàng 813 gửi từ huyện hoài hà nội hộp 4 quần lót nam boxer đùi thông hơi cao cấp quần sịp đùi nam dáng ôm sát body trẻ cuốn tạo cảm giác vô cùng thoải tự tin vận động tạo vẻ quyến rũ đầy nam tính quần có 3 size xxl chất liệu màu sắc có 3 màu đen tím hộp 4 chiếc đặc tính nổi bật quần sịp đùi nam dáng dài đến ôm sát body trẻ cuốn hút mà vẫn tạo cảm giác thoải thiết kế đặc biệt cho bạn tự tin vận tạo vẻ quyến rũ đầy nam tính cạp chun bản rộng co phù hợp với nhiều lứa không gây hằn trên da khử mùi co giãn 4 chiều tạo cảm giác thoải mái không bí cạp chun bản rộng co thoải mái không hằn trên da siêu thoáng mát cho anh em ngày hè nóng nực,hộp 4 sịp đùi boxer thông_hơi cao_cấp dành cho nam đồ_lót danh_mục shopee thời_trang nam đồ_lót quần_lót kiểu quần_lót boxers trunks kho hàng 813 gửi từ huyện hoài hà_nội hộp 4 quần_lót nam boxer đùi thông_hơi cao_cấp quần_sịp đùi nam_dáng ôm sát body trẻ cuốn tạo cảm_giác vô_cùng thoải tự_tin vận_động tạo vẻ quyến_rũ đầy nam_tính quần có 3 size xxl chất_liệu màu_sắc có 3 màu đen tím hộp 4 chiếc đặc_tính nổi_bật quần_sịp đùi nam_dáng dài đến ôm sát body trẻ cuốn_hút mà vẫn tạo cảm_giác thoải thiết_kế đặc_biệt cho bạn tự_tin vận_tạo vẻ quyến_rũ đầy nam_tính cạp_chun bản rộng_co phù_hợp với nhiều lứa không gây hằn trên da khử mùi co_giãn 4 chiều tạo cảm_giác thoải_mái không bí_cạp chun bản rộng co thoải_mái không hằn trên da siêu_thoáng mát cho anh_em ngày hè nóng_nực
22866,25527,Quần Lót Nam💞FREE SHIP💞Quần Sịp Đùi Nam Thông Hơi,Thời Trang Nam,Đồ Lót,https://shopee.vn/Qu%E1%BA%A7n-L%C3%B3t-Nam%F0%9F%92%9EFREE-SHIP%F0%9F%92%9EQu%E1%BA%A7n-S%E1%BB%8Bp-%C4%90%C3%B9i-Nam-Th%C3%B4ng-H%C6%A1i